**Fine-tuning GPT2** **Model**:

In [1]:
pip install transformers --upgrade


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 59.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 38.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 114.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 31.8 MB/s eta 0:00:00


In [2]:
!pip install torch
!pip install tqdm

In [3]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
#change directory path
import json
with open("/content/drive/MyDrive/Dataset/aug_train.json", "r") as f:
    train_data = json.load(f)


with open("/content/drive/MyDrive/Dataset/test.json", "r") as f:
    test_data = json.load(f)

with open("/content/drive/MyDrive/Dataset/val.json", "r") as f:
    val_data = json.load(f)

In [6]:
#change directory path
import torch
# Define a function to load data
import json
def load_from_json(filename):
    with open(filename, "r") as f:
        return torch.tensor(json.load(f))

# Load tokenized training data
loaded_train_inputs = load_from_json("/content/drive/MyDrive/Dataset/train_inputs.json")
loaded_train_targets = load_from_json("/content/drive/MyDrive/Dataset/train_targets.json")

# Load tokenized validation data
loaded_val_inputs = load_from_json("/content/drive/MyDrive/Dataset/val_inputs.json")
loaded_val_targets = load_from_json("/content/drive/MyDrive/Dataset/val_targets.json")

# Load tokenized test data
loaded_test_inputs = load_from_json("/content/drive/MyDrive/Dataset/test_inputs.json")
loaded_test_targets = load_from_json("/content/drive/MyDrive/Dataset/test_targets.json")


In [7]:
from torch.utils.data import DataLoader, TensorDataset

# Create TensorDatasets
train_dataset = TensorDataset(loaded_train_inputs, loaded_train_targets)
val_dataset = TensorDataset(loaded_val_inputs, loaded_val_targets)
test_dataset = TensorDataset(loaded_test_inputs, loaded_test_targets)

# Create DataLoaders
batch_size = 4  # You can adjust this value as needed
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)


In [11]:
from transformers import GPT2LMHeadModel, AdamW
# Load the GPT-2 model
model = GPT2LMHeadModel.from_pretrained('gpt2')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
# Optimizer
optimizer = AdamW(model.parameters(), lr=2e-5)


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [12]:
from transformers import GPT2Tokenizer

import torch

def process_data(data_subset):
    inputs = []
    targets = []
    for item in data_subset:
        input_text = f"Question: {item['question']} Choices: {' '.join([f'{key}. {value}' for key, value in item['choices'].items()])}"
        inputs.append(input_text)
        target_text = ','.join(item['correct_answer'])
        targets.append(target_text)
    return inputs, targets

train_inputs, train_targets = process_data(train_data)
val_inputs, val_targets = process_data(val_data)
test_inputs, test_targets = process_data(test_data)

tokenizer = GPT2Tokenizer.from_pretrained('gpt2-medium', padding_side='left')
tokenizer.pad_token = tokenizer.eos_token

# Tokenize the inputs
tokenized_train_inputs = tokenizer(train_inputs, truncation=True, padding=True, return_tensors='pt')
tokenized_val_inputs = tokenizer(val_inputs, truncation=True, padding=True, return_tensors='pt')
tokenized_test_inputs = tokenizer(test_inputs, truncation=True, padding=True, return_tensors='pt')

def tokenize_targets(targets_list, tokenizer, max_length=None):
    tokenized_targets = []
    max_len = 0

    # First, tokenize all targets
    for target in targets_list:
        tokenized_target = tokenizer(target, add_special_tokens=False)["input_ids"]
        tokenized_targets.append(tokenized_target)
        max_len = max(max_len, len(tokenized_target))

    # If max_length is not provided, use the maximum length among the tokenized targets
    max_length = max_length or max_len

    # Pad all tokenized targets to the same length
    padded_targets = []
    for tokenized_target in tokenized_targets:
        padded_target = tokenized_target + [tokenizer.pad_token_id] * (max_length - len(tokenized_target))
        padded_targets.append(padded_target)

    return torch.tensor(padded_targets)


# Tokenize the targets
tokenized_train_targets = tokenize_targets(train_targets, tokenizer)
tokenized_val_targets = tokenize_targets(val_targets, tokenizer)
tokenized_test_targets = tokenize_targets(test_targets, tokenizer)


In [13]:
#Base line model
#not actually need to run but if want to show on demo have to run
model.eval()
def predict_choice_key(model, tokenized_question, choices):
    """Predict the choice key based on logits."""
    max_logit = float('-inf')
    best_choice = None

    for key, choice in choices.items():
        if not choice:  # Skip empty choices
            continue

        input_text = tokenizer.decode(tokenized_question) + " " + choice
        tokenized_input = tokenizer.encode(
            input_text,
            return_tensors="pt",
            padding='max_length',
            max_length=MAX_LENGTH,
            truncation=True
        ).to(device)

        with torch.no_grad():
            outputs = model(tokenized_input)
            logits = outputs.logits[0]

        # Get the logit for the token corresponding to the choice
        tokenized_choice = tokenizer.encode(choice, add_special_tokens=False)
        if not tokenized_choice:  # Check if the choice is tokenized to a non-empty list
            continue

        choice_token_id = tokenized_choice[0]
        logit = logits[-1][choice_token_id].item()  # Convert tensor to scalar

        if logit > max_logit:
            max_logit = logit
            best_choice = key

    return best_choice

model.eval()  # Set the model to evaluation mode
MAX_LENGTH = 1024
correct_predictions = 0  # To count the number of correct predictions

def split_choices(choices_text):
    """Split the choices text into a dictionary."""
    keys = ['A', 'B', 'C', 'D', 'E']
    choices_dict = {}
    start = 0
    for i in range(1, len(keys)):
        end = choices_text.find(keys[i])
        choice_text = choices_text[start:end].replace(keys[i-1], '').strip()
        choices_dict[keys[i-1]] = choice_text.replace('.', '', 1).strip()  # Remove the period
        start = end
    choice_text = choices_text[start:].replace(keys[-1], '').strip()
    choices_dict[keys[-1]] = choice_text.replace('.', '', 1).strip()  # Remove the period
    return choices_dict


for idx in range(len(test_dataset)):
    tokenized_question, tokenized_correct_answer = test_dataset[idx]

    # Decode the correct answer
    decoded_correct_answer = tokenizer.decode(tokenized_correct_answer).replace("<|endoftext|>", "").strip()

    # Decode the entire tokenized_question
    full_decoded_text = tokenizer.decode(tokenized_question).strip()

    # Extract the choices part
    choices_text = full_decoded_text.split("Choices:", 1)[-1].strip()

    # Split the choices_text into individual choices
    choices_dict = split_choices(choices_text)

    predicted_answer_key = predict_choice_key(model, tokenized_question, choices_dict)  # Predict the answer key

    decoded_question = full_decoded_text.replace("Choices: " + choices_text, "").replace("Question: ", "", 1).replace("<|endoftext|>", "").strip()
    print("Question:", decoded_question)
    print("Choices:", choices_dict)
    print("Predicted Answer Key:", predicted_answer_key)
    print("Correct Answer Keys:", decoded_correct_answer)

    # Check if the predicted answer is one of the correct answers
    if predicted_answer_key == decoded_correct_answer:
        correct_predictions += 1

    print("-" * 50)

print(f"Accuracy: {correct_predictions / len(test_dataset) * 100:.2f}%")

Question: Which microorganism is responsible for the COVID-19 pandemic?
Choices: {'A': 'Influenza virus', 'B': 'Rhinovirus', 'C': 'SARS-oV-2', 'D': 'HIV', 'E': ''}
Predicted Answer Key: D
Correct Answer Keys: C
--------------------------------------------------
Question: What type of microorganism is SARS-CoV-2?
Choices: {'A': '', 'B': 'acteria  Virus', 'C': 'Fungus', 'D': 'Protozoa', 'E': ''}
Predicted Answer Key: D
Correct Answer Keys: B
--------------------------------------------------
Question: What is the primary site of SARS-CoV-2 replication in the human body?
Choices: {'A': 'Liver', 'B': 'lood', 'C': 'Respiratory tract', 'D': 'Kidney', 'E': ''}
Predicted Answer Key: B
Correct Answer Keys: C
--------------------------------------------------
Question: What type of test is primarily used to detect an active SARS-CoV-2 infection?
Choices: {'A': 'ntigen test', 'B': 'P', 'C': 'R test  Antibody test', 'D': 'Blood culture', 'E': ''}
Predicted Answer Key: B
Correct Answer Keys: B
----

In [14]:
from tqdm import tqdm

In [ ]:
import matplotlib.pyplot as plt


train_losses = []
val_losses = []
epochs = 5
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{epochs}"):
        input_ids, _ = batch
        labels = input_ids.clone()  # Set labels to be the same as input_ids
        input_ids, labels = input_ids.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(input_ids, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()
        train_loss = total_loss/len(train_loader)
        train_losses.append(train_loss)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}/{epochs} - Training Loss: {total_loss/len(train_loader)}")

    model.eval()
    vali_loss = 0
    with torch.no_grad():
        for batch in tqdm(val_loader, desc=f"Validation Epoch {epoch+1}/{epochs}"):
            input_ids, _ = batch
            labels = input_ids.clone()  # Set labels to be the same as input_ids
            input_ids, labels = input_ids.to(device), labels.to(device)
            outputs = model(input_ids, labels=labels)
            vali_loss += outputs.loss.item()
            val_loss = vali_loss/len(val_loader)
            val_losses.append(val_loss)
    print(f"Epoch {epoch+1}/{epochs} - Validation Loss: {vali_loss/len(val_loader)}")

# Save the Trained Model
save_directory = "/content/drive/MyDrive/GPT2/FinalDataset/gpt2_mediumTrained1epoch"
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

# Plotting the loss values
# Plotting the loss values
plt.figure(figsize=(10, 5))
plt.plot(train_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
plt.title('Training and Validation Losses')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.ylim([0, max(max(train_losses), max(val_losses)) + 0.2])  # Set y-axis limits
plt.legend()
plt.show()



In [15]:
#change directory path
# 5. Load the Saved trained Model
from transformers import GPT2LMHeadModel, AdamW, GPT2Tokenizer
import torch
load_directory = "/content/drive/MyDrive/Dataset/trained_model2"

# Load the saved model weights
model = GPT2LMHeadModel.from_pretrained(load_directory)
tokenizer = GPT2Tokenizer.from_pretrained(load_directory,padding_side='left')

# If you're using a GPU, move the model to the GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [16]:
model.eval()
test_loss = 0
with torch.no_grad():
    for batch in tqdm(test_loader, desc="Testing"):
        input_ids, _ = batch
        labels = input_ids.clone()  # Set labels to be the same as input_ids
        input_ids, labels = input_ids.to(device), labels.to(device)
        outputs = model(input_ids, labels=labels)
        test_loss += outputs.loss.item()
print(f"Test Loss: {test_loss/len(test_loader)}")


Testing: 100%|██████████| 18/18 [00:00<00:00, 18.85it/s]

Test Loss: 1.4571624663141038


In [17]:
def predict_choice_key(model, tokenized_question, choices):
    """Predict the choice key based on logits."""
    max_logit = float('-inf')
    best_choice = None

    for key, choice in choices.items():
        if not choice:  # Skip empty choices
            continue

        input_text = tokenizer.decode(tokenized_question) + " " + choice
        tokenized_input = tokenizer.encode(
            input_text,
            return_tensors="pt",
            padding='max_length',
            max_length=MAX_LENGTH,
            truncation=True
        ).to(device)

        with torch.no_grad():
            outputs = model(tokenized_input)
            logits = outputs.logits[0]

        # Get the logit for the token corresponding to the choice
        tokenized_choice = tokenizer.encode(choice, add_special_tokens=False)
        if not tokenized_choice:  # Check if the choice is tokenized to a non-empty list
            continue

        choice_token_id = tokenized_choice[0]
        logit = logits[-1][choice_token_id].item()  # Convert tensor to scalar

        if logit > max_logit:
            max_logit = logit
            best_choice = key

    return best_choice

model.eval()  # Set the model to evaluation mode
MAX_LENGTH = 1024
correct_predictions = 0  # To count the number of correct predictions

def split_choices(choices_text):
    """Split the choices text into a dictionary."""
    keys = ['A', 'B', 'C', 'D', 'E']
    choices_dict = {}
    start = 0
    for i in range(1, len(keys)):
        end = choices_text.find(keys[i])
        choice_text = choices_text[start:end].replace(keys[i-1], '').strip()
        choices_dict[keys[i-1]] = choice_text.replace('.', '', 1).strip()  # Remove the period
        start = end
    choice_text = choices_text[start:].replace(keys[-1], '').strip()
    choices_dict[keys[-1]] = choice_text.replace('.', '', 1).strip()  # Remove the period
    return choices_dict


for idx in range(len(test_dataset)):
    tokenized_question, tokenized_correct_answer = test_dataset[idx]

    # Decode the correct answer
    decoded_correct_answer = tokenizer.decode(tokenized_correct_answer).replace("<|endoftext|>", "").strip()

    # Decode the entire tokenized_question
    full_decoded_text = tokenizer.decode(tokenized_question).strip()

    # Extract the choices part
    choices_text = full_decoded_text.split("Choices:", 1)[-1].strip()

    # Split the choices_text into individual choices
    choices_dict = split_choices(choices_text)

    predicted_answer_key = predict_choice_key(model, tokenized_question, choices_dict)  # Predict the answer key

    decoded_question = full_decoded_text.replace("Choices: " + choices_text, "").replace("Question: ", "", 1).replace("<|endoftext|>", "").strip()
    print("Question:", decoded_question)
    print("Choices:", choices_dict)
    print("Predicted Answer Key:", predicted_answer_key)
    print("Correct Answer Keys:", decoded_correct_answer)

    # Check if the predicted answer is one of the correct answers
    if predicted_answer_key == decoded_correct_answer:
        correct_predictions += 1

    print("-" * 50)

print(f"Accuracy: {correct_predictions / len(test_dataset) * 100:.2f}%")

Question: Which microorganism is responsible for the COVID-19 pandemic?
Choices: {'A': 'Influenza virus', 'B': 'Rhinovirus', 'C': 'SARS-oV-2', 'D': 'HIV', 'E': ''}
Predicted Answer Key: C
Correct Answer Keys: C
--------------------------------------------------
Question: What type of microorganism is SARS-CoV-2?
Choices: {'A': '', 'B': 'acteria  Virus', 'C': 'Fungus', 'D': 'Protozoa', 'E': ''}
Predicted Answer Key: C
Correct Answer Keys: B
--------------------------------------------------
Question: What is the primary site of SARS-CoV-2 replication in the human body?
Choices: {'A': 'Liver', 'B': 'lood', 'C': 'Respiratory tract', 'D': 'Kidney', 'E': ''}
Predicted Answer Key: B
Correct Answer Keys: C
--------------------------------------------------
Question: What type of test is primarily used to detect an active SARS-CoV-2 infection?
Choices: {'A': 'ntigen test', 'B': 'P', 'C': 'R test  Antibody test', 'D': 'Blood culture', 'E': ''}
Predicted Answer Key: B
Correct Answer Keys: B
----

In [18]:
def predict_choice_keys(model, tokenized_question, choices):
    """Predict multiple choice keys based on logits."""
    logits_dict = {}

    for key, choice in choices.items():
        if not choice:  # Skip empty choices
            continue

        input_text = tokenizer.decode(tokenized_question) + " " + choice
        tokenized_input = tokenizer.encode(
            input_text,
            return_tensors="pt",
            padding='max_length',
            max_length=MAX_LENGTH,
            truncation=True
        ).to(device)

        with torch.no_grad():
            outputs = model(tokenized_input)
            logits = outputs.logits[0]

        # Get the logit for the token corresponding to the choice
        tokenized_choice = tokenizer.encode(choice, add_special_tokens=False)
        if not tokenized_choice:  # Check if the choice is tokenized to a non-empty list
            continue

        choice_token_id = tokenized_choice[0]
        logit = logits[-1][choice_token_id].item()  # Convert tensor to scalar
        logits_dict[key] = logit

    # Determine the threshold for prediction
    average_logit = sum(logits_dict.values()) / len(logits_dict)

    # Predict choices that have logits above the threshold
    predicted_keys = [key for key, logit in logits_dict.items() if logit > average_logit]

    return predicted_keys

model.eval()  # Set the model to evaluation mode
MAX_LENGTH = 1024
correct_predictions = 0  # To count the number of correct predictions

for idx in range(len(test_dataset)):
    tokenized_question, tokenized_correct_answer = test_dataset[idx]

    # Decode the correct answer
    decoded_correct_answer = tokenizer.decode(tokenized_correct_answer).replace("", "").replace("<|endoftext|>", "").strip()

    # Decode the entire tokenized_question
    full_decoded_text = tokenizer.decode(tokenized_question).strip()

    # Extract the choices part
    choices_text = full_decoded_text.split("Choices:", 1)[-1].strip()

    # Split the choices_text into individual choices
    choices_dict = split_choices(choices_text)

    predicted_answer_keys = predict_choice_keys(model, tokenized_question, choices_dict)  # Predict the answer keys

    decoded_question = full_decoded_text.replace("Choices: " + choices_text, "").replace("Question: ", "", 1).replace("", "").replace("<|endoftext|>", "").strip()
    print("Question:", decoded_question)
    print("Choices:", choices_dict)
    print("Predicted Answer Keys:", ', '.join(predicted_answer_keys))
    print("Correct Answer Keys:", decoded_correct_answer)

    # Check if the predicted answers match the correct answers
    if set(predicted_answer_keys) == set(decoded_correct_answer.split(',')):
        correct_predictions += 1

    print("-" * 50)

print(f"Accuracy: {correct_predictions / len(test_dataset) * 100:.2f}%")


Question: Which microorganism is responsible for the COVID-19 pandemic?
Choices: {'A': 'Influenza virus', 'B': 'Rhinovirus', 'C': 'SARS-oV-2', 'D': 'HIV', 'E': ''}
Predicted Answer Keys: C, D
Correct Answer Keys: C
--------------------------------------------------
Question: What type of microorganism is SARS-CoV-2?
Choices: {'A': '', 'B': 'acteria  Virus', 'C': 'Fungus', 'D': 'Protozoa', 'E': ''}
Predicted Answer Keys: C, D
Correct Answer Keys: B
--------------------------------------------------
Question: What is the primary site of SARS-CoV-2 replication in the human body?
Choices: {'A': 'Liver', 'B': 'lood', 'C': 'Respiratory tract', 'D': 'Kidney', 'E': ''}
Predicted Answer Keys: A, B, D
Correct Answer Keys: C
--------------------------------------------------
Question: What type of test is primarily used to detect an active SARS-CoV-2 infection?
Choices: {'A': 'ntigen test', 'B': 'P', 'C': 'R test  Antibody test', 'D': 'Blood culture', 'E': ''}
Predicted Answer Keys: B, C
Correct 

In [19]:
def predict_choice_keys(model, tokenized_question, choices):
    """Predict multiple choice keys based on logits."""
    logits_dict = {}

    for key, choice in choices.items():
        if not choice:  # Skip empty choices
            continue

        input_text = tokenizer.decode(tokenized_question) + " " + choice
        tokenized_input = tokenizer.encode(
            input_text,
            return_tensors="pt",
            padding='max_length',
            max_length=MAX_LENGTH,
            truncation=True
        ).to(device)

        with torch.no_grad():
            outputs = model(tokenized_input)
            logits = outputs.logits[0]

        # Get the logit for the token corresponding to the choice
        tokenized_choice = tokenizer.encode(choice, add_special_tokens=False)
        if not tokenized_choice:  # Check if the choice is tokenized to a non-empty list
            continue

        choice_token_id = tokenized_choice[0]
        logit = logits[-1][choice_token_id].item()  # Convert tensor to scalar
        logits_dict[key] = logit

    # Determine the threshold for prediction
    average_logit = sum(logits_dict.values()) / len(logits_dict)

    # Predict choices that have logits above the threshold
    predicted_keys = [key for key, logit in logits_dict.items() if logit > average_logit]

    return predicted_keys

model.eval()  # Set the model to evaluation mode
MAX_LENGTH = 1024
correct_predictions = 0  # To count the number of correct predictions

for idx in range(len(test_dataset)):
    tokenized_question, tokenized_correct_answer = test_dataset[idx]

    # Decode the correct answer
    decoded_correct_answer = tokenizer.decode(tokenized_correct_answer).replace("", "").replace("<|endoftext|>", "").strip()

    # Decode the entire tokenized_question
    full_decoded_text = tokenizer.decode(tokenized_question).strip()

    # Extract the choices part
    choices_text = full_decoded_text.split("Choices:", 1)[-1].strip()

    # Split the choices_text into individual choices
    choices_dict = split_choices(choices_text)

    predicted_answer_keys = predict_choice_keys(model, tokenized_question, choices_dict)  # Predict the answer keys

    decoded_question = full_decoded_text.replace("Choices: " + choices_text, "").replace("Question: ", "", 1).replace("", "").replace("<|endoftext|>", "").strip()
    print("Question:", decoded_question)
    print("Choices:", choices_dict)
    print("Predicted Answer Keys:", ', '.join(predicted_answer_keys))
    print("Correct Answer Keys:", decoded_correct_answer)

    # Check if the predicted answers match the correct answers
    if set(predicted_answer_keys) == set(decoded_correct_answer.split(',')):
        correct_predictions += 1

    print("-" * 50)

print(f"Accuracy: {correct_predictions / len(test_dataset) * 100:.2f}%")


Question: Which microorganism is responsible for the COVID-19 pandemic?
Choices: {'A': 'Influenza virus', 'B': 'Rhinovirus', 'C': 'SARS-oV-2', 'D': 'HIV', 'E': ''}
Predicted Answer Keys: C, D
Correct Answer Keys: C
--------------------------------------------------
Question: What type of microorganism is SARS-CoV-2?
Choices: {'A': '', 'B': 'acteria  Virus', 'C': 'Fungus', 'D': 'Protozoa', 'E': ''}
Predicted Answer Keys: C, D
Correct Answer Keys: B
--------------------------------------------------
Question: What is the primary site of SARS-CoV-2 replication in the human body?
Choices: {'A': 'Liver', 'B': 'lood', 'C': 'Respiratory tract', 'D': 'Kidney', 'E': ''}
Predicted Answer Keys: A, B, D
Correct Answer Keys: C
--------------------------------------------------
Question: What type of test is primarily used to detect an active SARS-CoV-2 infection?
Choices: {'A': 'ntigen test', 'B': 'P', 'C': 'R test  Antibody test', 'D': 'Blood culture', 'E': ''}
Predicted Answer Keys: B, C
Correct 

In [23]:
def predict_choice_demo(model, tokenized_question, choices):
    """Predict the choice key based on logits."""
    max_logit = float('-inf')
    best_choice = None

    for key, choice in choices.items():
        if not choice:  # Skip empty choices
            continue

        input_text = tokenizer.decode(tokenized_question[0]) + " " + choice  # Note the change here
        tokenized_input = tokenizer.encode(
            input_text,
            return_tensors="pt",
            padding='max_length',
            max_length=MAX_LENGTH,
            truncation=True
        ).to(device)

        with torch.no_grad():
            outputs = model(tokenized_input)
            logits = outputs.logits[0]

        # Get the logit for the token corresponding to the choice
        tokenized_choice = tokenizer.encode(choice, add_special_tokens=False)
        if not tokenized_choice:  # Check if the choice is tokenized to a non-empty list
            continue

        choice_token_id = tokenized_choice[0]
        logit = logits[-1][choice_token_id].item()  # Convert tensor to scalar

        if logit > max_logit:
            max_logit = logit
            best_choice = key

    return best_choice



In [26]:
def get_correct_answer_for_question(question_text, test_dataset):
    """Retrieve the correct answer for a given question from the test dataset."""
    for tokenized_question, tokenized_correct_answer in test_dataset:
        if question_text in tokenizer.decode(tokenized_question):
            return tokenizer.decode(tokenized_correct_answer).replace("", "").strip()
    return None

# Set the model to evaluation mode
model.eval()
MAX_LENGTH = 1024

# Input the question and choices from the user
question_text = input("Enter the question: ")
choices_text = input("Enter the choices (e.g., 'A. Choice 1 B. Choice 2 ...'): ")

# Split the choices_text into individual choices
choices_dict = split_choices(choices_text)

# Tokenize the input question
tokenized_question = tokenizer.encode(question_text, return_tensors="pt").to(device)

# Predict the answer key
predicted_answer_key = predict_choice_demo(model, tokenized_question, choices_dict)

# Fetch the correct answer from the test dataset
correct_answer = get_correct_answer_for_question(question_text, test_dataset).replace("", "").replace("<|endoftext|>", "").strip()

# Display the results
print("Question:", question_text)
print("Choices:", choices_dict)
print("Predicted Answer Key:", predicted_answer_key)
print("Correct Answer Keys:", correct_answer)


Enter the question: What family of viruses does SARS-CoV-2 belong to?
Enter the choices (e.g., 'A. Choice 1 B. Choice 2 ...'): A. Flaviviridae B. Paramyxoviridae C. oronaviridae D. Caliciviridae E.
Question: What family of viruses does SARS-CoV-2 belong to?
Choices: {'A': 'Flaviviridae', 'B': 'Paramyxoviridae', 'C': 'oronaviridae', 'D': 'Caliciviridae', 'E': ''}
Predicted Answer Key: C
Correct Answer Keys: C
